In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import torch
import copy
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import Tensor

In [2]:
train = pd.read_csv('dataAll.csv')
test = pd.read_csv('testAll.csv')
train_data=train.iloc[:,:30].applymap(lambda x: x+126.23)
train_loc = train.iloc[:,30:]

x_test = test.applymap(lambda x: x+126.23)
x_test = torch.from_numpy(
       x_test.values.astype(np.float32))

In [3]:
train_feat = torch.from_numpy(
        train_data.values.astype(np.float32))
train_labels = torch.from_numpy(
        train_loc.values.astype(np.float32))

In [4]:
num_train = int(0.8 * train_feat.shape[0]) # 划分训练样本和验证集样本
indices = np.arange( train_feat.shape[0])
np.random.shuffle(indices)  # shuffle 顺序
train_indices = indices[:num_train]
valid_indices = indices[num_train:]

train_features = train_feat[train_indices,:]
train_label = train_labels[train_indices,:]
valid_features  = train_feat[valid_indices,:]
valid_labels = train_labels[valid_indices,:]


In [5]:
feat_dim = train_data.shape[0]

In [6]:
def error(y_test,y_pre):
    y_test = np.array(y_test)
    y_pre = np.array(y_pre)
    m = y_test-y_pre
    m=np.square(m)
    m=m.sum(axis=1)
    m=np.sqrt(m)
    return m.sum()/len(y_test)

def get_model(feat_dim,output_dim = 2 ):
    net = nn.Sequential(
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim*4/5)),
        nn.BatchNorm1d(int(feat_dim*4/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*4/5),int(feat_dim*3/5)),
        nn.BatchNorm1d(int(feat_dim*3/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*3/5),int(feat_dim*2/5)),
        nn.BatchNorm1d(int(feat_dim*2/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*2/5),int(feat_dim/5)),
        nn.BatchNorm1d(int(feat_dim/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim/5), output_dim)
        )
    return net

def get_data(x,y,batch_size,shuffle):
    dataset = TensorDataset(x,y)
    return DataLoader(dataset, batch_size, shuffle=shuffle,num_workers=4)

def train_model(model,x_train,y_train,x_valid,y_valid,epochs,batch_size,lr,weight_decay,use_gpu):
    
    if use_gpu:
        model = model.cuda()
    train_data = get_data(x_train,y_train,batch_size,True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    #criterion = nn.CrossEntropyLoss()
    best_error = 0
    for e in  range(epochs):
        model.train()
        for data in train_data:
            x,y = data
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
                
            out = model(x)
            loss = criterion(out, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if use_gpu:
                x_valid = x_valid.cuda()
        with torch.no_grad():
            output = model(x_valid)
        output = Tensor.cpu(output)
        out = output.numpy()
        print('epoch:{}'.format(e))
        error_i = error(out,y_valid)
        print('error:{}'.format(error_i)) 
        if e == 0:
            best_error = error_i
        elif error_i < best_error:
            best_model = copy.deepcopy(model)
    torch.save(best_model,"model_NN_master.t7")

def predict(x_test,use_gpu):
    net = torch.load('./model_NN_master.t7')
    if use_gpu:
        x_test = x_test.cuda()
    with torch.no_grad():
        a = Tensor.cpu(net(x_test)).numpy()
    #a = net(x_test).detach().numpy()
    #a = np.argmax(a,axis=1)
    #np.savetxt('NN_ans.txt',np.array(a),fmt='%f')
    #print(a)
    a = pd.DataFrame(a)
    a.to_csv('NN_base.csv',header=None)
    '''
    with open('./NN_master_ans.txt','w') as f:
        for i in a:
            f.write('{}'.format(i))
            f.write('\n')'''
    

In [ ]:
# 可以调整的超参
batch_size = 128
epochs = 40000
lr = 0.001
wd = 0
use_gpu = False
feat_dim = train_feat.shape[1]
net = get_model(feat_dim)
train_model(net, train_features, train_label, valid_features, valid_labels, epochs, 
            batch_size, lr, wd, use_gpu)
predict(x_test,use_gpu)

epoch:0
error:37.0509429576326
epoch:1
error:36.966890362225094
epoch:2
error:36.87767322930142
epoch:3
error:36.78115145941138
epoch:4
error:36.67297915992885
epoch:5
error:36.56565835219922
epoch:6
error:36.44713423754851
epoch:7
error:36.29767343143596
epoch:8
error:36.10456925129366
epoch:9
error:35.89985496846701
epoch:10
error:35.67040952457956
epoch:11
error:35.384429576326
epoch:12
error:35.138724935316944
epoch:13
error:34.82012805223157
epoch:14
error:34.46441674078267
epoch:15
error:34.11365267221863
epoch:16
error:33.703564642626134
epoch:17
error:33.35397800776197
epoch:18
error:32.95247311610608
epoch:19
error:32.49379699628072
epoch:20
error:32.014131731080205
epoch:21
error:31.547787132115136
epoch:22
error:31.123327336675292
epoch:23
error:30.676943523609314
epoch:24
error:30.250517969760672
epoch:25
error:29.735097630983184
epoch:26
error:29.259146587968953
epoch:27
error:28.772570848156533
epoch:28
error:28.319951184508408
epoch:29
error:27.857737710219922
epoch:30
e

epoch:242
error:2.093623507994421
epoch:243
error:2.198979820656129
epoch:244
error:2.166955866449709
epoch:245
error:2.090747196646588
epoch:246
error:2.072680316643758
epoch:247
error:2.0468323622453104
epoch:248
error:2.0110368617601875
epoch:249
error:2.0856140847347997
epoch:250
error:2.1146428156330854
epoch:251
error:1.9391167604907826
epoch:252
error:2.010140363488438
epoch:253
error:2.0223374459290104
epoch:254
error:2.0758446697121604
epoch:255
error:2.0106960755578913
epoch:256
error:2.052954827983506
epoch:257
error:1.9620944044105757
epoch:258
error:1.9627475516453752
epoch:259
error:1.909696551837403
epoch:260
error:1.965201591304172
epoch:261
error:2.184764709077862
epoch:262
error:2.0601081798795278
epoch:263
error:2.050227748777086
epoch:264
error:1.9933965172218628
epoch:265
error:1.9793546412617238
epoch:266
error:1.920662192755498
epoch:267
error:1.9405817621583927
epoch:268
error:1.937154160434185
epoch:269
error:2.0408586548956986
epoch:270
error:2.006243430627425

epoch:478
error:1.722009735304819
epoch:479
error:1.7627016292043984
epoch:480
error:1.7934510303808215
epoch:481
error:1.7143080934670116
epoch:482
error:1.7265887143232534
epoch:483
error:1.6898269159827781
epoch:484
error:1.6783027204782504
epoch:485
error:1.741145243774256
epoch:486
error:1.6674206179758246
epoch:487
error:1.6434438293378073
epoch:488
error:1.8228091774134865
epoch:489
error:2.427879659201164
epoch:490
error:1.9605644986052717
epoch:491
error:1.9036203559589262
epoch:492
error:1.9347075429030562
epoch:493
error:1.7785434500828752
epoch:494
error:1.7065680776499839
epoch:495
error:1.6630537223075679
epoch:496
error:1.6883109069271507
epoch:497
error:1.7134943440228816
epoch:498
error:1.6835061057264715
epoch:499
error:1.7273955153925453
epoch:500
error:1.669481600703428
epoch:501
error:1.6586487684953104
epoch:502
error:1.6633360789941785
epoch:503
error:1.6701382221256469
epoch:504
error:1.7258123913526844
epoch:505
error:1.5999378436287193
epoch:506
error:1.628654

epoch:714
error:1.4992483121765847
epoch:715
error:1.5523834820706663
epoch:716
error:1.6233683952538809
epoch:717
error:1.6744907472913972
epoch:718
error:1.621113647618855
epoch:719
error:1.613049426948577
epoch:720
error:1.6002135677757114
epoch:721
error:1.7345245479766334
epoch:722
error:1.6176429343871281
epoch:723
error:1.5949035881407665
epoch:724
error:1.8022522525367886
epoch:725
error:2.0258046849733184
epoch:726
error:1.7122864322242886
epoch:727
error:1.5718214027631792
epoch:728
error:1.580883978563632
epoch:729
error:1.5672062606120634
epoch:730
error:1.693932931759379
epoch:731
error:1.8186573655299967
epoch:732
error:1.6099198157543662
epoch:733
error:1.6238528864812418
epoch:734
error:1.6184353649539134
epoch:735
error:1.6988460644404917
epoch:736
error:1.6634101423532504
epoch:737
error:1.7018141259197122
epoch:738
error:1.6367367526075356
epoch:739
error:1.6241565620451972
epoch:740
error:1.6185648573940814
epoch:741
error:1.5951175665022639
epoch:742
error:1.604592

epoch:950
error:1.5169181873079722
epoch:951
error:1.4570903111861255
epoch:952
error:1.4842009747938227
epoch:953
error:1.5017284082410252
epoch:954
error:1.4586827523649741
epoch:955
error:1.4851751685296732
epoch:956
error:1.503999895142707
epoch:957
error:1.5060015008489651
epoch:958
error:1.4348604071494988
epoch:959
error:1.5027152353856728
epoch:960
error:1.5072359428060318
epoch:961
error:1.4644053707147477
epoch:962
error:1.4677835442270375
epoch:963
error:1.5255670189703268
epoch:964
error:1.5124897037718306
epoch:965
error:1.5254117859597347
epoch:966
error:1.4892300190006469
epoch:967
error:1.5667734084431597
epoch:968
error:1.676176044024903
epoch:969
error:1.4812599172259056
epoch:970
error:1.6753280264594115
epoch:971
error:1.5147836149741267
epoch:972
error:1.558402827609557
epoch:973
error:1.5718847277247736
epoch:974
error:1.5366523614367724
epoch:975
error:1.5595769450396184
epoch:976
error:1.457433308234961
epoch:977
error:1.4963059912677878
epoch:978
error:1.545733

...

epoch:2100
error:1.4045037470690491
epoch:2101
error:1.3944884543277005
epoch:2102
error:1.4127421508631144
epoch:2103
error:1.4153138391413325
epoch:2104
error:1.5691176953630337
epoch:2105
error:1.8862754752688389
epoch:2106
error:1.7011838767383571
epoch:2107
error:1.5515522037718306
epoch:2108
error:1.5396148957996443
epoch:2109
error:1.476090168428606
epoch:2110
error:1.4903662362043175
epoch:2111
error:1.504145968931921
epoch:2112
error:1.4121731737144243
epoch:2113
error:1.4742687467153137
epoch:2114
error:1.4447555245896668
epoch:2115
error:1.5280995436812743
epoch:2116
error:1.460758421430304
epoch:2117
error:1.5109487437338291
epoch:2118
error:1.5582943382115135
epoch:2119
error:1.5673810754062905
epoch:2120
error:1.4513417943483182
epoch:2121
error:1.3928882751859637
epoch:2122
error:1.4322659787354464
epoch:2123
error:1.4322238147335866
epoch:2124
error:1.4132839661828913
epoch:2125
error:1.3948909863154917
epoch:2126
error:1.4251540012532342
epoch:2127
error:1.387333681173

epoch:2330
error:1.3537708198576972
epoch:2331
error:1.3587641746846701
epoch:2332
error:1.4134264078670764
epoch:2333
error:1.4212139568847024
epoch:2334
error:1.3752787245815814
epoch:2335
error:1.5251911750586191
epoch:2336
error:1.6078957857474936
epoch:2337
error:1.5541082582066623
epoch:2338
error:1.6390442447242886
epoch:2339
error:1.481802837968952
epoch:2340
error:1.4066760617116754
epoch:2341
error:1.4546980173128234
epoch:2342
error:1.6185691211695505
epoch:2343
error:1.5630092842921248
epoch:2344
error:1.4552662048734637
epoch:2345
error:1.4601638616288002
epoch:2346
error:1.415967302211352
epoch:2347
error:1.4316505738094276
epoch:2348
error:1.4340829787556597
epoch:2349
error:1.3991648685114813
epoch:2350
error:1.4296764457673028
epoch:2351
error:1.3960810534241592
epoch:2352
error:1.395882077235608
epoch:2353
error:1.4177173451649419
epoch:2354
error:1.387199135369502
epoch:2355
error:1.3945481471842658
epoch:2356
error:1.4176769182567917
epoch:2357
error:1.5868701614044

epoch:2560
error:1.4531395284201165
epoch:2561
error:1.3344265443078913
epoch:2562
error:1.3532980145334734
epoch:2563
error:1.4916024152550937
epoch:2564
error:1.370492083906048
epoch:2565
error:1.4499536985567594
epoch:2566
error:1.4541500432062582
epoch:2567
error:1.3817977214383894
epoch:2568
error:1.4113589505174644
epoch:2569
error:1.3213645471175615
epoch:2570
error:1.353804298391009
epoch:2571
error:1.3490176577154753
epoch:2572
error:1.380090316239489
epoch:2573
error:1.3912663034140524
epoch:2574
error:1.3295791053525226
epoch:2575
error:1.3112920882014067
epoch:2576
error:1.4181140342011642
epoch:2577
error:1.369742922764392
epoch:2578
error:1.3454097140806922
epoch:2579
error:1.3374538248908474
epoch:2580
error:1.3395893447303526
epoch:2581
error:1.391889920055789
epoch:2582
error:1.3391290148973156
epoch:2583
error:1.3170529226127912
epoch:2584
error:1.3888695878476713
epoch:2585
error:1.4004201555930627
epoch:2586
error:1.464461273548674
epoch:2587
error:1.382133928029996

epoch:2790
error:1.4178522068038486
epoch:2791
error:1.3893502890523932
epoch:2792
error:1.425179268070828
epoch:2793
error:1.372009040467335
epoch:2794
error:1.3785300902530724
epoch:2795
error:1.3747951808598804
epoch:2796
error:1.4065074057042368
epoch:2797
error:1.3580544929455045
epoch:2798
error:1.3849762870916882
epoch:2799
error:1.389257591415346
epoch:2800
error:1.437239120108344
epoch:2801
error:1.4139192687277653
epoch:2802
error:1.3813078610122898
epoch:2803
error:1.352892955863923
epoch:2804
error:1.4171292599854464
epoch:2805
error:1.387873759399256
epoch:2806
error:1.4795512486861255
epoch:2807
error:1.3992908867642302
epoch:2808
error:1.3352155006872575
epoch:2809
error:1.4086999338009378
epoch:2810
error:1.3425013454580368
epoch:2811
error:1.3508442907098965
epoch:2812
error:1.4490867308780724
epoch:2813
error:1.377669123443564
epoch:2814
error:1.3917421091728654
epoch:2815
error:1.415726951608991
epoch:2816
error:1.400825372180223
epoch:2817
error:1.4339811218972347
e

epoch:3020
error:1.4241273788708764
epoch:3021
error:1.465474630851795
epoch:3022
error:1.3602154375202133
epoch:3023
error:1.346008537657665
epoch:3024
error:1.3567573576972833
epoch:3025
error:1.3747691244542368
epoch:3026
error:1.381058351188551
epoch:3027
error:1.4268205057911545
epoch:3028
error:1.3396514063510672
epoch:3029
error:1.3495781073132276
epoch:3030
error:1.374329165992885
epoch:3031
error:1.4647319443321474
epoch:3032
error:1.3850186090111578
epoch:3033
error:1.4028841440612871
epoch:3034
error:1.3876005619340233
epoch:3035
error:1.354540194453428
epoch:3036
error:1.341614006407665
epoch:3037
error:1.3240345391079802
epoch:3038
error:1.40482574107576
epoch:3039
error:1.3430873777086028
epoch:3040
error:1.3186045420894648
epoch:3041
error:1.3330780858364328
epoch:3042
error:1.3509366725117238
epoch:3043
error:1.5541161540871604
epoch:3044
error:1.443632256630013
epoch:3045
error:1.3799751943018272
epoch:3046
error:1.3630431102441785
epoch:3047
error:1.3695966910575679
e

In [ ]:
y_valid = valid_labels#.cuda()
x_valid = valid_features.cuda()
net = torch.load('.//model_NN_master.t7')
with torch.no_grad():
    output_1 = net(x_valid)
output_1 = Tensor.cpu(output_1)  
out_1 = output_1.numpy()
error(out_1,y_valid)



In [25]:
predict(x_test,use_gpu)


In [11]:
from graphviz import Digraph
from torchviz import make_dot

In [19]:
make_dot(net(torch.randn(30,1)))


RuntimeError: size mismatch, m1: [30 x 1], m2: [30 x 30] at /opt/conda/conda-bld/pytorch_1549628766161/work/aten/src/TH/generic/THTensorMath.cpp:940